In [1]:
import pandas as pd

In [21]:
# load all our census data into dataframes
census_df = pd.read_csv('./csv/census_canada_scrape.csv')
doctor_count = pd.read_csv('./csv/doctor_city_count.csv')
closed_pharmacies_count = pd.read_csv('./csv/closed.csv')
open_pharmacies_count = pd.read_csv('./csv/combined.csv')

In [22]:
# Replace stoufville with more common name
census_df["City"] = census_df["City"].str.replace(
    "WhitchurchStouffville", "Stouffville"
)

# combine the doctor count with the census dataframe
for index, city in doctor_count.iterrows():
    census_df.loc[census_df["City"] == city[0], "Doctors"] = city[1]

# ratio of persons per doctor
census_df["Population, 2021 / Doctors"] = (
    census_df["Population, 2021"] / census_df["Doctors"]
)

# combine the doctor count with the census dataframe
for index, city in closed_pharmacies_count.iterrows():
    census_df.loc[census_df["City"] == city[0], "Closed Pharmacies"] = city[1]

census_df["Population, 2021 / Closed Pharmacies"] = (
    census_df["Population, 2021"] / census_df["Closed Pharmacies"]
)

# combine the doctor count with the census dataframe
for index, city in open_pharmacies_count.iterrows():
    census_df.loc[census_df["City"] == city[0], "Open Pharmacies"] = city[1]

# Calculate the ratio of persons per pharmacy
census_df["Population, 2021 / Open Pharmacies"] = (
    census_df["Population, 2021"] / census_df["Open Pharmacies"]
)
census_df["Population, 2026 / Open Pharmacies"] = (
    census_df["Population, 2026"] / census_df["Open Pharmacies"]
)

# Calculate the percentage of white collar workers
census_df["White Collar Workers"] = (
    census_df["51 Information and cultural industries"]
    + census_df["52 Finance and insurance"]
    + census_df["53 Real estate and rental and leasing"]
    + census_df["54 Professional, scientific and technical services"]
    + census_df["55 Management of companies and enterprises"]
    + census_df[
        "56 Administrative and support, waste management and remediation services"
    ]
    + census_df["61 Educational services"]
    + census_df["81 Other services (except public administration)"]
    + census_df["91 Public administration"]
)
census_df["White Collar Percentage"] = (
    census_df["White Collar Workers"] / census_df["All industries"] * 100
)

# drop unnecessary columns
census_df.drop(
    columns=[
        "Population, 2021",
        "Population, 2026",
        "Population, 2016",
        "White Collar Workers",
        "All industries",
        "51 Information and cultural industries",
        "52 Finance and insurance",
        "53 Real estate and rental and leasing",
        "54 Professional, scientific and technical services",
        "55 Management of companies and enterprises",
        "56 Administrative and support, waste management and remediation services",
        "61 Educational services",
        "62 Health care and social assistance",
        "81 Other services (except public administration)",
        "91 Public administration",
    ],
    inplace=True,
)
census_df.drop(index=[28, 46], inplace=True)

# fill in missing values
census_df["Closed Pharmacies"].fillna(0, inplace=True)
census_df["Population, 2021 / Closed Pharmacies"].fillna(999999, inplace=True)

In [38]:
positive_columns = census_df.columns[[1,2,3,4,5,6,7,8,9, 10, 11, 12, 14, 18, 19, 20]]
negative_columns = census_df.columns[[16]]

rank_df = pd.DataFrame()
rank_df['City'] = census_df['City']

for cols in positive_columns:
    rank_df[cols] = pd.qcut(census_df[cols], 10, labels=False, duplicates='drop') + 1
    
for cols in negative_columns:
    rank_df[cols] = 9 - pd.qcut(census_df[cols], 10, labels=False, duplicates='drop')

In [39]:
rank_df['Sum'] = rank_df.sum(axis=1, numeric_only=True)
rank_df['Sum Rank'] = rank_df['Sum'].rank(ascending=False)

In [40]:
rank_df[['City','Sum', 'Sum Rank']].to_csv('./csv/rank_locations.csv', index=False)